# Model Deployment

*Model formats, batch vs realtime predictions, and deployment pipelines*

---

Ethan Swan, 2023

# Agenda

1. About Me
2. Model Formats and Portability
3. Batch vs Realtime Scoring
4. Deployment Pipelines

# About Me

- Senior Backend Developer at [ReviewTrackers](https://www.reviewtrackers.com/)
    - Startup, ~100 employees
    - SaaS platform for online reputation management
- Analytics & ML Engineering Team
- NLP Microservice (Python), Main API Layer (Go)

- Started in February 2022
- Wanted to see a techy startup from the inside
    - especially engineering practices

# About Me

- Previously: [84.51˚](https://www.8451.com/)
    - Marketing Analytics Branch of Kroger
    - Lead Data Scientist - Internal Tools & Infrastructure
- Education: University of Notre Dame
    - B.S. in Computer Science
    - M.B.A.

- 8451:
    - Did some measurement work
    - quickly transitioned to functional support
    - taught classses and helped with tech strategy

# ...